# Harmonic oscillator in a visous liquid with thermal ziggling

In [1]:
import numpy as np
import matplotlib.pyplot as plt

class HarmonicOscillatorLangevin:
    def __init__(self, k=1.0, m=1.0, gamma=0.1, T=300.0, x0=1.0, v0=0.0):
        """Initialize the Langevin oscillator with given parameters."""
        self.k = k
        self.m = m
        self.gamma = gamma
        self.T = T
        self.k_B = 1 #1.380649e-23 - Think about the concept of reduced unit
        self.x = x0
        self.v = v0
        self.time_evolution_strategy = None

    def set_time_evolution_strategy(self, strategy):
        """Set the strategy for time evolution."""
        self.time_evolution_strategy = strategy

    def compute_force(self):
        """Compute the deterministic part of the force for the Langevin oscillator."""
        return -self.k * self.x

    def time_evolution(self, dt, num_steps):
        """Simulate the time evolution using the specified strategy for a given number of steps."""
        return self.time_evolution_strategy.evolve(self, dt, num_steps)
    
    def plot_phase_space_trajectory(self, x_values, v_values):
        """Plot the phase space trajectory of the Langevin oscillator."""
        plt.figure(figsize=(8, 6))
        plt.plot(x_values, v_values, label='Phase Space Trajectory')
        plt.xlabel('Position x')
        plt.ylabel('Velocity v')
        plt.title('Phase Space Trajectory using Langevin Dynamics')
        plt.legend()
        plt.grid(True)
        plt.show()
        
    def plot_position_vs_time(self, times, x_values):
        """Plot the position as a function of time for the Langevin oscillator."""
        plt.figure(figsize=(8, 6))
        plt.plot(times, x_values, label='Position vs Time')
        plt.xlabel('Time')
        plt.ylabel('Position x')
        plt.title('Position using Langevin Dynamics')
        plt.legend()
        plt.grid(True)
        plt.show()

    def plot_velocity_vs_time(self, times, v_values):
        """Plot the velocity as a function of time for the Langevin oscillator."""
        plt.figure(figsize=(8, 6))
        plt.plot(times, v_values, label='Velocity vs Time')
        plt.xlabel('Time')
        plt.ylabel('Velocity v')
        plt.title('Velocity using Langevin Dynamics')
        plt.legend()
        plt.grid(True)
        plt.show()


# Here is a code for time evolution
You need to edit """LanveginVelocityVerletMethod""".

In [2]:
###
# Time evolution
###
class EulerMethod:
    @staticmethod
    def evolve(oscillator, dt, num_steps):
        times = np.linspace(0, dt*num_steps, num_steps+1)
        x_values, v_values = [oscillator.x], [oscillator.v]
        
        for _ in range(num_steps):
            oscillator.v += oscillator.compute_force() / oscillator.m * dt - oscillator.gamma * oscillator.v * dt
            oscillator.x += oscillator.v * dt
            
            x_values.append(oscillator.x)
            v_values.append(oscillator.v)
        
        return times, x_values, v_values


class EulerMaruyamaMethod:
    @staticmethod
    def evolve(oscillator, dt, num_steps):
        times = np.linspace(0, dt*num_steps, num_steps+1)
        x_values, v_values = [oscillator.x], [oscillator.v]
        
        for _ in range(num_steps):
            deterministic_force = oscillator.compute_force()
            stochastic_force = np.sqrt(2. * oscillator.gamma * oscillator.k_B * oscillator.T) * np.random.normal()
            
            oscillator.v += (deterministic_force / oscillator.m - oscillator.gamma * oscillator.v) * dt + stochastic_force / oscillator.m * np.sqrt(dt)
            oscillator.x += oscillator.v * dt
            
            x_values.append(oscillator.x)
            v_values.append(oscillator.v)
        
        return times, x_values, v_values

    
# This is the part you need to EDIT!!!
# Velocity Verlet Method
class LangevinVelocityVerletMethod:
    @staticmethod
    def evolve(oscillator, dt, num_steps):
        times = np.linspace(0, dt*num_steps, num_steps+1)
        x_values, v_values = [oscillator.x], [oscillator.v]
        
        sqrt_dt = np.sqrt(2 * oscillator.gamma * oscillator.k_B * oscillator.T / oscillator.m) * np.sqrt(dt)
        
        for _ in range(num_steps):
            # Half update of velocity with deterministic and stochastic forces
            a = oscillator.compute_force() / oscillator.m
            v_half = oscillator.v + 0.5 * dt * (a - oscillator.gamma * oscillator.v) + sqrt_dt * np.random.normal(0, 1)
            
            # Update position
            oscillator.x += # EDIT HERE

            # New acceleration
            a_new = # EDIT HERE

            # Complete velocity update
            oscillator.v = v_half + 0.5 * dt * (a_new - oscillator.gamma * oscillator.v) 

            x_values.append(oscillator.x)
            v_values.append(oscillator.v)
        
        return times, x_values, v_values

SyntaxError: invalid syntax (1302144935.py, line 55)

# Run your simulation

In [ ]:
# Create an instance and set the evolution strategy
oscillator = HarmonicOscillatorLangevin(gamma=0.1, T=0.01)

# Create an instance and set the evolution strategy
oscillator.set_time_evolution_strategy(LangevinVelocityVerletMethod)
times_verlet, x_values_verlet, v_values_verlet = oscillator.time_evolution(dt=0.01, num_steps=1000000)

# plot
oscillator.plot_phase_space_trajectory(x_values_verlet, v_values_verlet)
oscillator.plot_position_vs_time(times_verlet, x_values_verlet)
oscillator.plot_velocity_vs_time(times_verlet, v_values_verlet)


# Theoretical prediction from equipartition theorem
theoretical_KE = oscillator.k_B * oscillator.T / 2

# Calculate the kinetic energy over time
KE_values_diff = [0.5 * oscillator.m * v**2 - theoretical_KE for v in v_values_verlet]

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(times_verlet, KE_values_diff, label='Simulated KE', alpha=0.7)
#plt.axhline(y=theoretical_KE, color='r', linestyle='-', label='Theoretical KE')
plt.xlabel('Time')
plt.ylabel('Kinetic Energy from Theory')
plt.legend()
plt.title('Kinetic Energy as a function of time')
plt.grid(True)
plt.show()

# Calculate the potential energy over time
PE_values = [0.5 * oscillator.k * x**2 for x in x_values_verlet]

# Calculate the kinetic energy over time
KE_values = [0.5 * oscillator.m * v**2 for v in v_values_verlet]

# Calculate the average potential and kinetic energies
avg_PE = sum(PE_values[1000:]) / len(PE_values[1000:])
avg_KE = sum(KE_values[1000:]) / len(KE_values[1000:])

# Print out the results
print("Average Potential Energy:", avg_PE)
print("Average Kinetic Energy:", avg_KE, theoretical_KE)
